In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


- 설계
 - CNN1 : 5x5x1 32개 (28)
 - max pool : (14)
 - CNN2 : 5x5x32 64개 (14)
 - max pool : (7)
 
 - FC1 (1024)
 - FC2 (10)

In [12]:
# paramater
learning_rate = 0.01
epochs = 1000
batch_size = 256
disp_step = 100

# layer parameter
n_input = 28 * 28 # 28 * 28
n_output = 10
dropout = 0.75

weights = {
    'wc1' : tf.Variable(tf.random_normal([5, 5, 1, 32])), # 5x5,1 channel filter 32 개
    'wc2' : tf.Variable(tf.random_normal([5, 5, 32, 64])), # 5x5,32 channel filter 64 개
    'wd1' : tf.Variable(tf.random_normal([7*7*64, 1024])), # 7*7*64, 1024개
    'wd2' : tf.Variable(tf.random_normal([1024, n_output])) # 1024 -> 10
}

biases = {
    'bc1' : tf.Variable(tf.random_normal([32])),
    'bc2' : tf.Variable(tf.random_normal([64])),
    'bd1' : tf.Variable(tf.random_normal([1024])),
    'bd2' : tf.Variable(tf.random_normal([n_output]))
}

In [39]:
#  define input, output
X = tf.placeholder(tf.float32, [None, n_input])
Y = tf.placeholder(tf.float32, [None, n_output])
keep_prob = tf.placeholder(tf.float32)

##### conv2d 사용법
`tf.nn.conv2d(input, filter, strides, padding)`
- `input` : [batch, i_h, i_w, i_ch]
- `filter` : [f_h, f_w, i_ch, o_ch]
- `strides` : 크기 4인 1차원 리스트 [1, stride, stride, 1]
- `padding` : 'SAME', 'VALID'

##### max_pool 사용법
`tf.nn.max_pool(value, ksize, strides, padding)`
- `value` : [batch, i_h, i_w, i_ch]
- `ksize` : 크기가 4인 1차원 리스트 [i, f_h, f_w, f_ch]
- `stride` : 크기 4인 1차원 리스트 [1, stride, stride, 1]

##### dropout
- keep_prob = train에는 placeholder 값으로 설정, test에는 1로 설정

In [40]:
# define model
def cnn(x):
    x = tf.reshape(x, [-1, 28, 28, 1])
    
    # CNN1 (28*28*1) -> (14*14*32)
    conv1 = tf.nn.conv2d(x, weights['wc1'], strides = [1, 1, 1, 1], padding = 'SAME')
    conv1 = tf.nn.relu(tf.nn.bias_add(conv1, biases['bc1']))
    conv1 = tf.nn.max_pool(conv1, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
    
    # CNN2 (14*14*32) -> (7*7*64)
    conv2 = tf.nn.conv2d(conv1, weights['wc2'], strides = [1, 1, 1, 1], padding = 'SAME')
    conv2 = tf.nn.relu(tf.nn.bias_add(conv2, biases['bc2']))
    conv2 = tf.nn.max_pool(conv2, ksize = [1, 2, 2, 1], strides=[1, 2, 2, 1], padding = 'SAME')
    
    # FC (7*7*64) -> (10)
    fc1 = tf.reshape(conv2, [-1, 7*7*64])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    
    fc1 = tf.nn.dropout(fc1, keep_prob)
    
    # out
    out = tf.add(tf.matmul(fc1, weights['wd2']), biases['bd2'])
    
    return out

In [41]:
# get logits and predicts
logits = cnn(X)
predicts = tf.nn.softmax(logits)

In [42]:
weights['wd1'].get_shape().as_list()

[3136, 1024]

In [43]:
# define loss
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = logits, labels = Y))

In [44]:
# define optimizer
optimizer = tf.train.AdamOptimizer()
train_op = optimizer.minimize(loss_op)

In [45]:
# evaluation (accuracy)
correct = tf.equal(tf.argmax(predicts, 1), tf.argmax(Y, 1))
acc_op = tf.reduce_mean(tf.cast(correct, tf.float32))

In [46]:
init = tf.global_variables_initializer()

##### Run

In [50]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(epochs):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        
        sess.run(train_op, feed_dict={X : batch_x, Y : batch_y, keep_prob: 0.75})
        
        if epoch % disp_step == 0:
            loss, acc = sess.run([loss_op, acc_op], feed_dict={X: batch_x, Y: batch_y, keep_prob: 1.0})
            print("Step " + str(epoch) + ", Minibatch Loss= " + "{:.4f}".format(loss) + ", Training Accuracy= " + "{:.3f}".format(acc))
            
    print("finish optimizer")
        
    print("Testing Accuracy:", \
    sess.run(acc_op, feed_dict={X: mnist.test.images[:256],
                                  Y: mnist.test.labels[:256],
                                  keep_prob: 1.0}))

Step 0, Minibatch Loss= 83002.4766, Training Accuracy= 0.051
Step 100, Minibatch Loss= 2036.8503, Training Accuracy= 0.887
Step 200, Minibatch Loss= 1202.4476, Training Accuracy= 0.922
Step 300, Minibatch Loss= 784.4600, Training Accuracy= 0.938
Step 400, Minibatch Loss= 697.4205, Training Accuracy= 0.961
Step 500, Minibatch Loss= 238.1519, Training Accuracy= 0.957
Step 600, Minibatch Loss= 339.7564, Training Accuracy= 0.945
Step 700, Minibatch Loss= 360.9007, Training Accuracy= 0.965
Step 800, Minibatch Loss= 447.6774, Training Accuracy= 0.945
Step 900, Minibatch Loss= 34.0279, Training Accuracy= 0.988
finish optimizer
Testing Accuracy: 0.980469
